In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Cargamos las tablas normalizadas (archivos creados en el proceso de ETL)

In [4]:
items_games = pd.read_parquet('..\\Archivos API\\items_games_API.parquet')
reviews_games = pd.read_parquet('..\\Archivos API\\reviews_games.parquet')

In [5]:
items_games

,genres,year_release,playtime_forever,user_id
0,Action,2000,6,76561197970982479
1,Action,2000,0,js41637
2,Action,2000,0,Riot-Punch
3,Action,2000,93,doctr
4,Action,2000,108,corrupted_soul
...,...,...,...,...
7461478,Indie,2016,46,76561198071808318
7461479,Action,2016,164,76561198107283457
7461480,Casual,2016,164,76561198107283457
7461481,Indie,2016,164,76561198107283457


In [6]:
reviews_games

,user_id,posted,item_id,recommend,sentiment_analysis,year_posted,title,genres,release_date,year_release
0,76561197970982479,2011-11-05,1250,1,2,2011,Killing Floor,Action,2009-05-14,2009
1,death-hunter,2015-03-30,1250,1,2,2015,Killing Floor,Action,2009-05-14,2009
2,DJKamBer,2013-07-12,1250,1,0,2013,Killing Floor,Action,2009-05-14,2009
3,diego9031,2015-08-13,1250,1,1,2015,Killing Floor,Action,2009-05-14,2009
4,76561198081962345,2014-04-05,1250,1,1,2014,Killing Floor,Action,2009-05-14,2009
...,...,...,...,...,...,...,...,...,...,...
101582,Gamer0009,2015-08-11,306040,1,2,2015,The Howler,Simulation,2016-04-21,2016
101583,Gamer0009,2015-08-11,306040,1,2,2015,The Howler,Strategy,2016-04-21,2016
101584,llDracuwulf,2015-10-29,307130,1,2,2015,Asteria,Action,2014-07-03,2014
101585,llDracuwulf,2015-10-29,307130,1,2,2015,Asteria,Adventure,2014-07-03,2014


Vemos cuáles son los distintos géneros

In [7]:
generos = items_games['genres'].unique()
generos

array(['Action', 'RPG', 'Strategy', 'Casual', 'Simulation', 'Indie',
       'Racing', 'Sports', 'Adventure', 'Early Access', 'Free to Play',
       'Massively Multiplayer', 'Utilities', 'Animation &amp; Modeling',
       'Video Production', 'Design &amp; Illustration', 'Web Publishing',
       'Education', 'Software Training', 'Photo Editing',
       'Audio Production'], dtype=object)

# **FUNCIONES**

1) PlayTimeGenre

In [8]:
def PlayTimeGenre(genero: str):
    '''
    Devuelve el año con más horas jugadas para el género de entrada

    Args: 
        genero (str): género de juego del que se quiere averiguar cuál fue el año en el que más se jugó.

    return:
        dict: diccionario donde la clave es el género y el valor es el año con más horas jugadas.
    
    '''
    # Filtramos el dataframe 'items_games' por género
    df_util = items_games[items_games['genres']== genero]
    
    # Agrupamos el dataframe anterior por año de lanzamiento, suma de minutos de juego y ordenamos en forma descendente
    df_agrupado = df_util.groupby('year_release')['playtime_forever'].sum().sort_values(ascending=False)

    # Como están ordenados de manera descendente, el valor máximo tendrá el índice [0]
    anio = df_agrupado.index[0]
    
    return {f"Año de lanzamiento con más horas jugadas para el género {genero}": int(anio)}

In [9]:
genero = PlayTimeGenre('Strategy')
genero

{'Año de lanzamiento con más horas jugadas para el género Strategy': 2015}

2) UserForGenre

In [10]:
def UserForGenre(genero: str):
    '''
    Devuelve el usuario con más minutos jugados para el género de entrada

    Args: 
        genero (str): género de juego del que se quiere averiguar cuál fue el usuario con más horas jugadas.

    return:
        dict: diccionario donde hay dos valores:el usuario con más horas jugadas para ese género, y la cantidad total de horas que jugó cada año.
    
    '''
    # Filtramos el dataframe 'items_games' respecto al parámetro genero
    df_util = items_games[items_games['genres']== genero]

    # Agrupamos el dataframe anterior por usuario, sumamos los minutos de juego y ordenamos en forma descendente
    df_agrupado = df_util.groupby('user_id')['playtime_forever'].sum().sort_values(ascending=False)

    # Como están ordenados de manera descendente, el valor máximo tendrá el índice [0]
    user = df_agrupado.index[0]

    # Tomamos las filas del dataframe util que contengan su repsectivo usuario (user)
    df_genero_user = df_util[df_util['user_id']==user]

    # Agrupamos respecto a los años y suma de minutos de juego
    minutos_jugados = round(df_genero_user.groupby('year_release')['playtime_forever'].sum(), 3)

    # Guardamos la serie 'minutos_jugados' en una lista
    lista_minutos_jugados = [f'Año: {int(anio)}, Horas: {horas}' for anio, horas in minutos_jugados.items()]

    return {f"Usuario con más horas jugadas para género {genero}": user, "Horas jugadas": lista_minutos_jugados}

In [11]:
usuario = UserForGenre('Action')
usuario

{'Usuario con más horas jugadas para género Action': 'REBAS_AS_F-T',
 'Horas jugadas': ['Año: 1996, Horas: 0',
  'Año: 1998, Horas: 2',
  'Año: 1999, Horas: 225',
  'Año: 2000, Horas: 0',
  'Año: 2001, Horas: 11',
  'Año: 2002, Horas: 1',
  'Año: 2003, Horas: 1863',
  'Año: 2004, Horas: 2115',
  'Año: 2005, Horas: 3293',
  'Año: 2006, Horas: 138',
  'Año: 2007, Horas: 375',
  'Año: 2008, Horas: 2573',
  'Año: 2009, Horas: 7926',
  'Año: 2010, Horas: 4460',
  'Año: 2011, Horas: 37705',
  'Año: 2012, Horas: 50635',
  'Año: 2013, Horas: 97566',
  'Año: 2014, Horas: 158114',
  'Año: 2015, Horas: 162452',
  'Año: 2016, Horas: 138572',
  'Año: 2017, Horas: 1990']}

3) UserRecommend

In [12]:
def UsersRecommend(anio: int):
    '''
    Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado.
    
    Args: 
        anio (int): año del que se quiere saber el top 3 juegos más recomendados.

    return:
        dict: diccionario con los 3 juegos más recomendados por usuarios
        ->  reviews['recommend'] = True(1)
        ->  comentarios positivos(2) o neutros(1)

    '''
    # Si el tipo de dato ingresado no es de tipo entero, retorna un mensaje de error.
    if type(anio) != int:
        return f"El formato del dato ingresado es incorrecto, se espera una entrada del tipo 'int'"
    
    else:
        # Si el año de entrada no correasponde a algún año de los que se hizo reseña (year_posted), retorna un mensaje de error.
        if anio not in reviews_games['year_posted'].unique():
            return f"El año ingresado se encuentra fuera del rango de registros, intente nuevamente"
        
        else:
            # Filtramos el dataframe por las filas donde el año de posteo (year_posted) no es menor al año de publicación (year_release).
            df = reviews_games[reviews_games['year_posted']>=reviews_games['year_release']]
            
            # Filtramos el dataframe 'df' para el año parámetro y donde la columna 'sentiment_analysis' sea positiva (2) o neutra (1)
            df_anio = df[(df['year_posted']==anio) & (df['sentiment_analysis'].isin([1,2]))]

            # Agrupamos el dataframe 'df_anio' por el título del juego (title), sumamos las recomendaciones (recommend) para obtener los juegos más recomendados, y ordenamos de forma descendente.
            top = df_anio.groupby('title')['recommend'].sum().sort_values(ascending=False)

            # Construimos el top 3 de juegos más recomendados por los usuarios.
            top_tres = [{"Puesto 1": top.index[0]}, {"Puesto 2": top.index[1]}, {"Puesto 3": top.index[2]}]

    return top_tres

In [13]:
top_3 = UsersRecommend(2011)
top_3

[{'Puesto 1': 'Team Fortress 2'},
 {'Puesto 2': 'Terraria'},
 {'Puesto 3': 'Portal 2'}]

4) UsersNotRecommend

In [14]:
def UsersNotRecommend(anio: int):
    '''
    Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado
    
    Args: 
        anio (int): año del que se quiere saber el top 3 juegos menos recomendados.

    return:
        dict: diccionario con los 3 juegos menos recomendados por usuarios
        ->  reviews['recommend'] = False(0)
        ->  comentarios negativos(reviews['sentiment_analysis']==0)

    '''
    # Si el tipo de dato ingresado no es de tipo entero, retorna un mensaje de error.
    if type(anio) != int:
        return f"El formato del dato ingresado es incorrecto, se espera una entrada del tipo 'int'"
    
    else:
        # Si el año de entrada no correasponde a algún año de los que se hizo reseña (year_posted), retorna un mensaje de error.
        if anio not in reviews_games['year_posted'].unique():
            return f"El año ingresado se encuentra fuera del rango de registros, intente nuevamente"
        
        else:
            # Filtramos el dataframe por las filas donde el año de posteo (year_posted) no es menor al año de publicación (year_release).
            df = reviews_games[reviews_games['year_posted']>=reviews_games['year_release']]

            # Filtramos el dataframe 'df' para el año de entrada y donde las columnas 'recommend' y 'sentiment_analysis' sean negativas (0) 
            df_anio = df[(df['year_posted']==anio) & (df['recommend']==0) & (df['sentiment_analysis']==0)]

            # Agrupamos el dataframe 'df_anio' por el título del juego (title), contamos las recomendaciones (recommend) para obtener los juegos con más recomendaciones negativas, y ordenamos de forma descendente.
            top = df_anio.groupby('title')['recommend'].count().sort_values(ascending=False)

            # Construimos el top 3 de juegos más recomendados por los usuarios.
            top_tres = [{"Puesto 1": top.index[0]}, {"Puesto 2": top.index[1]}, {"Puesto 3": top.index[2]}]

    return top_tres

In [15]:
top_3 = UsersNotRecommend(2011)
top_3

[{'Puesto 1': 'And Yet It Moves'},
 {'Puesto 2': 'Team Fortress 2'},
 {'Puesto 3': 'Men of War: Vietnam'}]

5) sentiment_analysis

In [16]:
def sentiment_analysis(anio: int):
    '''
    Según el año de lanzamiento, devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento

    Args: 
        anio (int): año del que se quiere saber la cantidad de registros de reseñas de usuarios.

    return:
        list: lista con la cantidad de registros de reseñas de usuarios en ese año. Las cantidades están separadas por tipo de reseña (negativa, positiva y neutra)

    '''
    # Si el tipo de dato ingresado no es de tipo entero, retorna un mensaje de error.
    if type(anio) != int:
        return f"El formato del dato ingresado es incorrecto, se espera una entrada del tipo 'int'"
    
    else:
        # Si el año de entrada no correasponde a algún año de los que se hizo reseña (year_posted), retorna un mensaje de error.
        if anio not in reviews_games['year_posted'].unique():
            return f"El año ingresado se encuentra fuera del rango de registros, intente nuevamente"
        
        else:
            # Filtramos el dataframe con las filas cuyo año de posteo (year_posted) no es menor al año de publicación (year_release).
            df = reviews_games[reviews_games['year_posted']>=reviews_games['year_release']]

            # Filtramos el dataframe 'df' para el año de entrada.
            df_anio = df[df['year_release'] == anio]

            # Contamos las filas del dataframe 'df_anio' respecto a los valores únicos de la columna 'sentiment_analysis' (0, 1 y 2) y los agrupamos de acuerdo a su categoría.
            positive = df_anio[df_anio['sentiment_analysis']==2].shape[0]  # Número de filas de sentimientos positivos (2)
            neutral = df_anio[df_anio['sentiment_analysis']==1].shape[0]   # Número de filas de sentimientos neutros (1)
            negative = df_anio[df_anio['sentiment_analysis']==0].shape[0]  # Número de filas de sentimientos negativos (0)

    return {'Negative': negative, 'Neutral': neutral, 'Positive': positive}

In [17]:
sentimientos = sentiment_analysis(2010)
sentimientos

{'Negative': 486, 'Neutral': 523, 'Positive': 2005}